# Hate Speech Detection using HateBERT

This notebook implements a hate speech detection model using **HateBERT**, a BERT model retrained on abusive language communities from Reddit. This approach leverages transfer learning to achieve high performance on hate speech classification tasks.

## Workflow Overview
1. **Setup**: Install dependencies and configure the environment.
2. **Preprocessing**: Apply advanced text cleaning tailored for social media text.
3. **Data Loading**: Load and filter the `hate_speech_offensive` dataset.
4. **Augmentation**: Use the EasyMix strategy to augment minority classes.
5. **Splitting**: Create stratified train, validation, and test splits.
6. **Tokenization**: Tokenize text using the BERT tokenizer.
7. **Class Balancing**: Implement a weighted loss function to handle class imbalance.
8. **Model**: Initialize and fine-tune the HateBERT model.
9. **Training**: Train the model with early stopping.
10. **Evaluation**: Evaluate using accuracy, F1-score, and confusion matrix.
11. **Inference**: Predict classes for new inputs.

## 1. Installation & Setup
Install the required Python packages, including `transformers`, `datasets`, and `torch`.

In [1]:
# Install required packages
!pip install transformers datasets torch scikit-learn tqdm imbalanced-learn nlpaug

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.5/410.5 kB 15.6 MB/s eta 0:00:00


## 2. Import Libraries & Configuration
Import necessary libraries and set up the configuration class for hyperparameters and device selection.

In [2]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    EarlyStoppingCallback
)
from datasets import load_dataset, concatenate_datasets
from sklearn.metrics import (
    accuracy_score,
    f1_score,
    precision_recall_fscore_support,
    classification_report,
    confusion_matrix
)
from sklearn.utils.class_weight import compute_class_weight
import re
from tqdm.auto import tqdm
import warnings
warnings.filterwarnings('ignore')

# Set random seeds for reproducibility
SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

# Configuration
class Config:
    MODEL_NAME = "bert-base-uncased"  # Can also use: roberta-base, distilbert-base-uncased
    MAX_LENGTH = 128
    BATCH_SIZE = 16
    LEARNING_RATE = 2e-5
    NUM_EPOCHS = 5
    WARMUP_STEPS = 500
    WEIGHT_DECAY = 0.01
    NUM_LABELS = 3
    DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

config = Config()
print(f"Using device: {config.DEVICE}")

Using device: cuda


## 3. Advanced Text Preprocessing
Define a preprocessing function to clean social media text by removing URLs, mentions, and excessive punctuation, and handling repeated characters.

In [3]:
def advanced_preprocess(text):
    """
    Advanced preprocessing for social media text
    """
    if not isinstance(text, str):
        return ""

    # Convert to lowercase
    text = text.lower()

    # Remove URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '[URL]', text, flags=re.MULTILINE)

    # Replace mentions with token
    text = re.sub(r'@\w+', '[USER]', text)

    # Replace hashtags but keep content
    text = re.sub(r'#(\w+)', r'\1', text)

    # Handle repeated characters (e.g., "coooool" -> "cool")
    text = re.sub(r'(.)\1{2,}', r'\1\1', text)

    # Remove excessive punctuation
    text = re.sub(r'([!?.]){2,}', r'\1', text)

    # Remove extra whitespace
    text = re.sub(r'\s+', ' ', text).strip()

    # Remove very short texts (likely noise)
    if len(text.split()) < 3:
        return ""

    return text

## 4. Data Loading & Cleaning
Load the dataset and apply the cleaning function. Filter out empty or very short samples.

In [4]:
# Load dataset
dataset = load_dataset("tdavidson/hate_speech_offensive")

# Filter and preprocess
def filter_and_clean(examples):
    """Filter out empty/invalid samples and clean text"""
    cleaned_texts = []
    valid_labels = []

    for text, label in zip(examples['tweet'], examples['class']):
        cleaned = advanced_preprocess(text)
        if cleaned and len(cleaned) > 10:  # Keep only meaningful text
            cleaned_texts.append(cleaned)
            valid_labels.append(label)

    return {
        'text': cleaned_texts,
        'label': valid_labels
    }

# Process dataset
processed_dataset = dataset['train'].map(
    filter_and_clean,
    batched=True,
    remove_columns=dataset['train'].column_names
)

print(f"Samples after preprocessing: {len(processed_dataset)}")
print(f"Class distribution: {np.bincount([x for x in processed_dataset['label']])}")

README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/1.63M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/24783 [00:00<?, ? examples/s]

Map:   0%|          | 0/24783 [00:00<?, ? examples/s]

Samples after preprocessing: 24555
Class distribution: [ 1401 19029  4125]


## 5. Data Augmentation (EasyMix Strategy)
Augment the minority classes to balance the dataset. This implementation uses a simplified augmentation strategy involving random word deletion to simulate variety.

In [5]:
def augment_minority_classes(dataset, target_samples_per_class=7000):
    """
    Augment minority classes using back-translation simulation
    """
    from collections import Counter
    import random

    # Count samples per class
    label_counts = Counter(dataset['label'])
    print(f"Original distribution: {label_counts}")

    augmented_texts = []
    augmented_labels = []

    for label in label_counts.keys():
        # Get samples for this class
        class_samples = [
            (text, lbl) for text, lbl in zip(dataset['text'], dataset['label'])
            if lbl == label
        ]

        current_count = len(class_samples)
        needed = max(0, target_samples_per_class - current_count)

        # Add original samples
        for text, lbl in class_samples:
            augmented_texts.append(text)
            augmented_labels.append(lbl)

        # Augment if needed
        if needed > 0:
            print(f"Augmenting class {label}: adding {needed} samples")
            for _ in range(needed):
                # Random sampling with replacement
                original_text, original_label = random.choice(class_samples)

                # Simple word-level augmentation
                words = original_text.split()
                if len(words) > 3:
                    # Random word deletion (10% chance per word)
                    words = [w for w in words if random.random() > 0.1]

                augmented_text = ' '.join(words) if words else original_text
                augmented_texts.append(augmented_text)
                augmented_labels.append(original_label)

    # Create new dataset
    from datasets import Dataset
    augmented_dataset = Dataset.from_dict({
        'text': augmented_texts,
        'label': augmented_labels
    })

    print(f"Augmented distribution: {Counter(augmented_dataset['label'])}")
    return augmented_dataset

# Apply augmentation
balanced_dataset = augment_minority_classes(processed_dataset, target_samples_per_class=6000)

Original distribution: Counter({1: 19029, 2: 4125, 0: 1401})
Augmenting class 2: adding 1875 samples
Augmenting class 0: adding 4599 samples
Augmented distribution: Counter({1: 19029, 2: 6000, 0: 6000})


## 6. Train/Val/Test Split
Split the balanced dataset into training (70%), validation (15%), and test (15%) sets, ensuring stratified sampling to maintain class distribution.

In [6]:
from sklearn.model_selection import train_test_split

# Get indices
indices = list(range(len(balanced_dataset)))
labels = balanced_dataset['label']

# Stratified split: 70% train, 15% validation, 15% test
train_idx, temp_idx = train_test_split(
    indices, test_size=0.3, random_state=SEED, stratify=labels
)

temp_labels = [labels[i] for i in temp_idx]
val_idx, test_idx = train_test_split(
    temp_idx, test_size=0.5, random_state=SEED, stratify=temp_labels
)

# Create splits
train_dataset = balanced_dataset.select(train_idx)
val_dataset = balanced_dataset.select(val_idx)
test_dataset = balanced_dataset.select(test_idx)

print(f"Train: {len(train_dataset)} | Val: {len(val_dataset)} | Test: {len(test_dataset)}")

Train: 21720 | Val: 4654 | Test: 4655


## 7. Tokenization
Tokenize the text data using the pre-trained BERT tokenizer, applying padding and truncation to a maximum length.

In [7]:
# Initialize tokenizer
tokenizer = AutoTokenizer.from_pretrained(config.MODEL_NAME)

def tokenize_function(examples):
    """Tokenize texts with padding and truncation"""
    return tokenizer(
        examples['text'],
        padding='max_length',
        truncation=True,
        max_length=config.MAX_LENGTH,
        return_tensors='pt'
    )

# Tokenize datasets
train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

# Set format for PyTorch
train_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
val_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
test_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/21720 [00:00<?, ? examples/s]

Map:   0%|          | 0/4654 [00:00<?, ? examples/s]

Map:   0%|          | 0/4655 [00:00<?, ? examples/s]

## 8. Weighted Loss Function
Calculate class weights and define a custom Trainer with a weighted Cross-Entropy loss function to further address any remaining class imbalance during training.

In [8]:
# Compute class weights
train_labels = [example['label'].item() for example in train_dataset]
class_weights = compute_class_weight(
    'balanced',
    classes=np.unique(train_labels),
    y=train_labels
)
class_weights = torch.tensor(class_weights, dtype=torch.float).to(config.DEVICE)

print(f"Class weights: {class_weights}")

# Custom Trainer with weighted loss
class WeightedLossTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits

        # Weighted CrossEntropyLoss
        loss_fct = nn.CrossEntropyLoss(weight=class_weights)
        loss = loss_fct(logits, labels)

        return (loss, outputs) if return_outputs else loss

Class weights: tensor([1.7238, 0.5435, 1.7238], device='cuda:0')


## 9. Model Initialization
Load the pre-trained `GroNLP/hateBERT` model for sequence classification.

In [9]:
config.MODEL_NAME = "GroNLP/hateBERT"

# Reload everything with HateBERT
tokenizer = AutoTokenizer.from_pretrained(config.MODEL_NAME)
model = AutoModelForSequenceClassification.from_pretrained(
    config.MODEL_NAME,
    num_labels=config.NUM_LABELS,
    problem_type="single_label_classification"
).to(config.DEVICE)

print(f"Model loaded: {config.MODEL_NAME}")
print(f"Total parameters: {sum(p.numel() for p in model.parameters()):,}")

tokenizer_config.json:   0%|          | 0.00/151 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at GroNLP/hateBERT and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model loaded: GroNLP/hateBERT
Total parameters: 109,484,547


## 10. Evaluation Metrics
Define a function to compute accuracy, precision, recall, and F1-score (weighted and macro) for model evaluation.

In [10]:
def compute_metrics(eval_pred):
    """Compute comprehensive metrics"""
    predictions, labels = eval_pred
    preds = np.argmax(predictions, axis=1)

    # Calculate metrics
    accuracy = accuracy_score(labels, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, preds, average='weighted', zero_division=0
    )
    f1_macro = f1_score(labels, preds, average='macro')

    return {
        'accuracy': accuracy,
        'f1_weighted': f1,
        'f1_macro': f1_macro,
        'precision': precision,
        'recall': recall
    }

## 11. Training Configuration
Set up the training arguments and initialize the Trainer with the model, datasets, and early stopping callback.

In [11]:
training_args = TrainingArguments(
    output_dir='./results',
    eval_strategy='epoch',
    save_strategy='epoch',
    learning_rate=config.LEARNING_RATE,
    per_device_train_batch_size=config.BATCH_SIZE,
    per_device_eval_batch_size=config.BATCH_SIZE,
    num_train_epochs=config.NUM_EPOCHS,
    weight_decay=config.WEIGHT_DECAY,
    warmup_steps=config.WARMUP_STEPS,
    logging_dir='./logs',
    logging_steps=100,
    load_best_model_at_end=True,
    metric_for_best_model='f1_weighted',
    greater_is_better=True,
    save_total_limit=2,
    fp16=torch.cuda.is_available(),  # Mixed precision training
    dataloader_num_workers=2,
    report_to='none'  # Disable wandb/tensorboard
)

# Initialize trainer with early stopping
trainer = WeightedLossTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

## 12. Train the Model
Execute the training process.

In [12]:
print("🚀 Starting training...")
trainer.train()
print("✅ Training complete!")

🚀 Starting training...


Epoch,Training Loss,Validation Loss,Accuracy,F1 Weighted,F1 Macro,Precision,Recall
1,0.400600,0.330251,0.875376,0.878864,0.860879,0.890123,0.875376
2,0.212000,0.304230,0.905028,0.907148,0.893309,0.914474,0.905028
3,0.133100,0.349890,0.919639,0.920954,0.907695,0.925383,0.919639
4,0.055800,0.376105,0.938333,0.938499,0.927326,0.938974,0.938333
5,0.048800,0.388690,0.938547,0.938786,0.927659,0.939376,0.938547


✅ Training complete!


## 13. Evaluation on Test Set
Evaluate the trained model on the held-out test set and print detailed classification reports and the confusion matrix.

In [13]:
# Evaluate on test set
print("\n📊 Evaluating on test set...")
test_results = trainer.evaluate(test_dataset)

print("\n=== Test Set Results ===")
for key, value in test_results.items():
    if not key.startswith('eval_'):
        continue
    metric_name = key.replace('eval_', '').upper()
    print(f"{metric_name}: {value:.4f}")

# Detailed classification report
predictions = trainer.predict(test_dataset)
preds = np.argmax(predictions.predictions, axis=1)
true_labels = predictions.label_ids

print("\n=== Detailed Classification Report ===")
target_names = ['Hate Speech', 'Offensive Language', 'Neither']
print(classification_report(true_labels, preds, target_names=target_names, digits=4))

# Confusion Matrix
print("\n=== Confusion Matrix ===")
cm = confusion_matrix(true_labels, preds)
print(cm)


📊 Evaluating on test set...



=== Test Set Results ===
LOSS: 0.3872
ACCURACY: 0.9353
F1_WEIGHTED: 0.9357
F1_MACRO: 0.9241
PRECISION: 0.9365
RECALL: 0.9353
RUNTIME: 8.2861
SAMPLES_PER_SECOND: 561.7810
STEPS_PER_SECOND: 35.1190

=== Detailed Classification Report ===
                    precision    recall  f1-score   support

       Hate Speech     0.8741    0.9333    0.9027       900
Offensive Language     0.9627    0.9408    0.9516      2855
           Neither     0.9159    0.9200    0.9180       900

          accuracy                         0.9353      4655
         macro avg     0.9176    0.9314    0.9241      4655
      weighted avg     0.9365    0.9353    0.9357      4655


=== Confusion Matrix ===
[[ 840   49   11]
 [ 104 2686   65]
 [  17   55  828]]


## 14. Save Model
Save the fine-tuned model and tokenizer for future use.

In [14]:
# Save best model
model.save_pretrained('./best_hate_speech_model')
tokenizer.save_pretrained('./best_hate_speech_model')
print("✅ Model saved to './best_hate_speech_model'")

✅ Model saved to './best_hate_speech_model'


## 15. Inference Function
Define a function to perform inference on single text inputs and display the prediction confidence.

In [15]:
def predict_hate_speech(text, model, tokenizer, device=config.DEVICE):
    """
    Predict hate speech for a single text
    """
    # Preprocess
    cleaned_text = advanced_preprocess(text)

    # Tokenize
    inputs = tokenizer(
        cleaned_text,
        padding='max_length',
        truncation=True,
        max_length=config.MAX_LENGTH,
        return_tensors='pt'
    ).to(device)

    # Predict
    model.eval()
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        probs = torch.softmax(logits, dim=1)
        prediction = torch.argmax(probs, dim=1).item()
        confidence = probs[0][prediction].item()

    labels = {0: 'Hate Speech', 1: 'Offensive Language', 2: 'Neither'}

    return {
        'text': text,
        'prediction': labels[prediction],
        'confidence': confidence,
        'probabilities': {
            'Hate Speech': probs[0][0].item(),
            'Offensive Language': probs[0][1].item(),
            'Neither': probs[0][2].item()
        }
    }

# Test inference
test_texts = [
    "I love spending time with my friends!",
    "This movie was absolutely terrible",
    "You are such an idiot"
]

print("\n=== Sample Predictions ===")
for text in test_texts:
    result = predict_hate_speech(text, model, tokenizer)
    print(f"\nText: {result['text']}")
    print(f"Prediction: {result['prediction']} (Confidence: {result['confidence']:.2%})")
    print(f"Probabilities: {result['probabilities']}")


=== Sample Predictions ===

Text: I love spending time with my friends!
Prediction: Neither (Confidence: 99.97%)
Probabilities: {'Hate Speech': 0.0001368294470012188, 'Offensive Language': 0.000154142253450118, 'Neither': 0.9997090697288513}

Text: This movie was absolutely terrible
Prediction: Neither (Confidence: 99.77%)
Probabilities: {'Hate Speech': 0.0004538965586107224, 'Offensive Language': 0.0018721850356087089, 'Neither': 0.9976739287376404}

Text: You are such an idiot
Prediction: Neither (Confidence: 93.16%)
Probabilities: {'Hate Speech': 0.013980165123939514, 'Offensive Language': 0.05446198210120201, 'Neither': 0.9315578937530518}
